# ASTR 21100 

# "Computational Techniques in Astrophysics"

## Instructor: Andrey Kravtsov

### office: ERC 415; email: kravtsov@uchicago.edu
### office hours: Tue, 10:30am-noon (unless noted otherwise)

#### Teaching Assistants: 
#### Dimitrios Tanoglidis (dtanoglidis@uchicaago.edu)
#### Georgios Zakharegkas (gzakharegkas@uchicago.edu)

In [1]:
import numpy as np 

# use jupyter "magic" command to tell it to embed plot into the notebook 
import matplotlib.pyplot as plt
%matplotlib inline
# reusing my codes from the repository
from codes.plot_utils import plot_pretty
plot_pretty(dpi=150, fontsize=12)

In [2]:
plot_pretty(dpi=150, fontsize=12)

### Sampling random numbers from a probability distribution function (PDF)

This is by far the most common task in Monte Carlo applications. Such sampling involves two ingredients: 

1) a sequence of random, uniformly distributed numbers in the $[a,b]$ interval (this is usually handled by a pseudo-random generator routine);

2) a transformation of that sequence from the uniform PDF to a desired PDF using some method; 

### Random sampling from pdfs

The two simplest and widely used methods to sample non-uniform pdfs -- the inverse transform sampling and rejection sampling -- were laid out on one page in a letter by John von Neumann to Stanislaw Ulam written on May 21, 1947. 

The latter was known previously in a different context (geometric estimates of probability), but it was von Neumann who re-invented rejection sampling and made it popular in the context of Monte Carlo methods. 

<img width=600 src="http://astro.uchicago.edu/~andrey/classes/a330f17/fig/von_neumann_letter1.png"></img>

### Inverse transform sampling

*``Assume that you want one [random sample] with the distribution function (density) $f(\xi)d\xi$: $\{\xi_i\}$. One way to form it is to form the cumulative distribution function: $x = F(\xi)=\int^\xi_0f(\xi)d\xi$, to invert it $F^{-1}(x) =\xi$, $x=F(\xi)$, and 
to form $\xi_i=F^{-1}(x_i)$ with this $F^{-1}(x)$, or some approximant polynomial.''*  where ${x_i}$ are uniformly distributed numbers on the relevant interval. 

"Invert" here literally means "find a function that solves for $\xi$ given $x=F(\xi)$." For some pdfs $F^{-1}(x)$ can be derived analytically, while for many others it can be solved for numerically for each $x$. 

Von Neumann in the quote above suggests inverting in such cases by approximating $x(F)$ by an "approximant polynomial." 
Good choices for these are global polynomial interpolation using Chebyshev nodes (but never equidistant nodes!) or *splines*.   

### Inverse transform sampling: geometric interpretation

The meaning of the method can be grasped by visualizing the cumulative pdf in the $\xi-x$ plane and considering uniform 
distribution of $x$ in the equal size intervals along the ordinate axis and imagining how these would map to the abscissa-axis of the $\xi$ variable. 

The mapping is clearly provided by the local slope of $F(\xi)$ - i.e., $dF/d\xi=f(\xi)$. 


In [ ]:
from numpy import random as rnd

Nrnd = 100000
# generate a vector of Nrnd uniformly distributed numbers over [0.,1.]
xrnd = rnd.uniform(0., 1., Nrnd)

# write a routine that transforms it into exponential pdf p_exp(y) = exp(-y/beta)/beta
# generate random numbers using it and plot their distribution to check that it is correct


### Rejection sampling

If the maximum, $p_{\rm max}$, of the pdf $p(x)$ in the interval $x\in [a, b]$ is known, the random samples of $p(x)$ can be generated by drawing two random numbers $x$ and $y$ uniformly distributed in the intervals $[a,b]$ and $[0,p_{\rm max}]$, respectively. $x$ is then accepted as a sample of $p(x)$ if $p(x)>y$. If $p(x)<y$ the draw is rejected, and the process is repeated until the sample of accepted draws of $x$ reaches desired size.

### Rejection sampling: geometric interpretation

The logic of this method is best grasped visually: we are throwing "darts" into random locations in the $(x,y)$ plane and accept only those locations that fall under the curve $p(x)$. The result will clearly be the set of $x$ that have distribution $p(x)$.  

In [ ]:
# write a code sampling exponential pdf p_exp(y) = exp(-y/beta)/beta
# above using rejection method

from numpy import random as rnd

Nrnd = 100000
# generate a vector of Nrnd uniformly distributed numbers over [0.,1.]
xrnd = rnd.uniform(0., 30., Nrnd)
yrnd = rnd.uniform(0.,1./beta, Nrnd)

Plot the resulting distribution to check that it is correct. 

What fraction of samples is rejected? 

### Special cases: drawing Gaussian random numbers

The Gaussian numbers can be drawn using the <a href="https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform">Box-Mueller method.</a>. 

If $u_1$ and $u_2$ are uniformly distributed random numbers in $(0,1)$, the following two numbers are drawn from the Gaussian pdf of zero mean and unit variance:


$$g_1=\sqrt{-2\ln u_1}\cos 2\pi u_2;\ \ \ g_2=\sqrt{-2\ln u_1}\sin 2\pi u_2 $$

Using Gaussian samples you can also create samples of some other distributions, such as $\chi^2$ and Student $t$ distribution (see notes for details).

Overall, though, it is more efficient to use the <a href="https://en.wikipedia.org/wiki/Ziggurat_algorithm">*ziggurat algorithm*</a> of Marsaglia & Tang (2000) to sample symmetric unimodal distributions such as Gaussian and Student $t$. However, this algorithm won't work for assymetric distributions like $\chi^2$.  


### Importance sampling

Often we are interested not in the samples from a given distribution $p(x)$ themselves, but rather in averages of various quantities over that distribution:

$$
\bar{g}_{p} = \int\limits_a^{b}g(x)p(x)dx \approx \frac{1}{N_s}\sum\limits_{i=1}^{N_s} g(x_{i,p}),\ \ \mathrm{where}\ \ x_{i,p}\sim p(x),
$$
where the last part indicates that $x_{i,p}$ are samples drawn from $p(x)$.

If this is the case and $p(x)$ is a distribution that's difficult to sample from, we can choose a function, $q(x)$, called the *importance* function, that's easy to sample from, and re-write the expression for $\bar{g}_p$ as

$$
\bar{g}_{p} = \int\limits_a^{b}g(x)\,\frac{p(x)}{q(x)}\, q(x)dx \approx \frac{1}{N_s}\sum\limits_{i=1}^{N_s} g(x_{i,q})\,\frac{p(x_{i,q})}{q(x_{i,q})},\ \ \mathrm{where}\ \ x_{i,q}\sim q(x).
$$

Thus, we can get an estimate of an average (or any other moment) over a given distribution, by sampling from a different distribution. Note that for numerical stability reasons, $p/q$ ratio should never be overly large. Thus $q(x)$ is chosen in such a way that $p(x)/q(x)$ goes to zero or varies mildly as $p(x)$ goes to zero. 

The importance sampling is useful not only because it allows to compute averages over distribution by sampling an easier to sample distribution, but also because
such averages can be more accurate than the straightforward average over $p(x)$. This is because sampling from $q(x)$ can concentrate the samples into *important* regions that contribute the most to the integral. 